In [88]:
%pip -q install google-genai PyPDF2 google-adk

In [89]:
# Configura a API Key do Google Gemini

import os
import glob
from google.colab import userdata
import logging

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [90]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [91]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [92]:
#Excelente para extrair texto, tabelas e informações de layout.
import pdfplumber
import PyPDF2
logging.getLogger("pdfminer.pdfpage").setLevel(logging.ERROR)

In [93]:
print("🚀 Iniciando o Sistema Gerador de Lista de Compras Necessárias 🚀")

🚀 Iniciando o Sistema Gerador de Lista de Compras Necessárias 🚀


In [94]:
def extract_text_from_pdf_pdfplumber(pdf_path):
    """Extrai texto de um PDF usando pdfplumber."""
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n" # Adiciona nova linha entre páginas
        #print(f"Texto extraído com pdfplumber (primeiros 500 chars): {text[:500]}...")
        return text
    except Exception as e:
        print(f"Erro ao extrair texto com pdfplumber: {e}")
        return None

In [95]:
def get_text_all_pdf():

  text_all = ""
  datas = ""
  caminho_da_pasta = "/content/compras"
  # Constrói o padrão de busca por arquivos .pdf dentro da pasta
  # os.path.join garante que o caminho funcione em diferentes sistemas operacionais
  padrao_busca = os.path.join(caminho_da_pasta, "*.pdf")
  # Usa glob para encontrar todos os arquivos que correspondem ao padrão
  arquivos_pdf = glob.glob(padrao_busca)
  if not arquivos_pdf:
        print(f"Nenhum arquivo PDF encontrado na pasta: {caminho_da_pasta}")
        return

  print(f"Encontrados {len(arquivos_pdf)} arquivos PDF na pasta {caminho_da_pasta}:")

  for caminho_arquivo in arquivos_pdf:
    #print(f"\n--- Lendo arquivo: {os.path.basename(caminho_arquivo)} ---") # os.path.basename pega só o nome do arquivo
    try:
        # Abre o arquivo PDF em modo binário de leitura ('rb')
        with open(caminho_arquivo, 'rb') as arquivo_pdf:

            #Pega a data da compra no título
            data_compra = os.path.basename(caminho_arquivo).replace(".pdf", "")

            text_all = text_all + "\nItens comprados no dia " + data_compra + "\n"

            if(datas!=""):
              datas = datas + ", "

            datas = datas + data_compra

            # Cria um objeto PdfReader
            texto = extract_text_from_pdf_pdfplumber(caminho_arquivo)

            text_all = text_all + texto

            text_all = text_all + "fim da compra\n"

    except FileNotFoundError:
        print(f"  Erro: Arquivo não encontrado {caminho_arquivo}. Isso não deveria acontecer com glob.")
    except Exception as e:
        print(f"  Ocorreu um erro inesperado ao processar {os.path.basename(caminho_arquivo)}: {e}")

  #print(datas)
  #print(text_all)
  return datas, text_all


In [96]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [97]:
################################################
# --- Agente 1: resumidor de compras --- #
# --- lê todos os pdfs de lista de compras realizadas --- #
# --- e gera uma lista com todos os itens, totalizando a--- #
# --- quantidade comprada, media de consumo por dia e--- #
# --- a data da ultima compra--- #
################################################
def agente_resumidor_compras(historico, intervalo):
    planejador = Agent(
        name="agente_resumidor_compras",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Basealo no histórico de compras
        Gere uma lista contendo o Nome do Item, quatidade total comprada,
        média por dia (quatidade total comprada dividida pelo Intervalo entre compas calculado na resposta anterior)
        e a ultima data de compra, no formaro Nome - Quatidade comprada - Média por Dia - Ultima Compra
        """,
        description="Agente que lê o histórico de compas e gera um resumo",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Histórico de compras:{historico}\nIntervalo entre compras: {intervalo}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [98]:
################################################
# --- Agente 1: calculador intervalo em dias --- #
# --- lê o título de todos os pdfs e calculo o intervalo --- #
# ---  em dias entre a primeira e ultima compra--- #
################################################
def agente_calculador_intervalo(datas):
    planejador = Agent(
        name="agente_calculador_intervalo",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        utilize as datas fornecidas para calcular o intervalo em dias entre a menor e maior data
        A sua resposta deve ser apenas o intervalo
        """,
        description="Agente que calcula o intervalo em dias entre a menor e maior data",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Datas:{datas}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [99]:
################################################
# --- Agente 1: gerador próxima lista de compra --- #
################################################
def agente_gerador_lista_compras(resumo_compras,quatidade_dias_proxima_compra):
    planejador = Agent(
        name="agente_gerador_lista_compras",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        baseado no resumo de compras
        considerando a media de consumo de cada item
        gere uma lista de itens que devem ser comprados para suprir o estoque para os próximos dia de abastecimento
        """,
        description="Agente que gera a lista dos itens que precisam de abastecimento suprindo a necessidade dos próximos quatidade_dias_proxima_compra dias",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Datas:{resumo_compras}\nDias de abastecimento: {quatidade_dias_proxima_compra}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
quatidade_dias_proxima_compra = input("A sua próxima compra é para abastecer quantos dias?")

datas_compras, historico_compras = get_text_all_pdf()

intervalo_entre_compras = agente_calculador_intervalo(datas_compras)


resumo = agente_resumidor_compras(historico_compras,intervalo_entre_compras)

lista_compras_para_hoje = agente_gerador_lista_compras(resumo,quatidade_dias_proxima_compra)

print(lista_compras_para_hoje)

A sua próxima compra é para abastecer quantos dias?15
Encontrados 15 arquivos PDF na pasta /content/compras:
